# Visualizing a Wire Plane Image Tutorial

This Tutorial Notebook shows how to view the input wire plane images. We also explain the contents of the image a little and define some jargon used by the group and by the collaboration.

You can get an example file from:

https://drive.google.com/drive/folders/1LOOLHZJYLmOZAiGT4IAgmZwgUGTRjBdD?usp=sharing

By default, the notebook will reference the following file:

`dlmerged_larflowtruth_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root`

This is simulated data. It contains simulated inputs that go into the reconstruction workflow. 
It also contains, "truth" information saved from the simulation that helps us generate labels and study the performance of the reconstruction and quantify the accuracy of physics analyses based.

In this notebook you will see:
* The class we use to interface with this type of file
* Some utility functions we have built, based on the plotly graphics library, to do visualization

------

Some classes and functions used in this tutorial:

* [larcv::IOManager](https://github.com/LArbys/LArCV/blob/develop/larcv/core/DataFormat/IOManager.h)
* [larcv::Image2D](https://github.com/LArbys/LArCV/blob/develop/larcv/core/DataFormat/Image2D.h)
* [plotly heatmap](https://plotly.com/python/heatmaps/)
* [lardly image2d plot makeing function](https://github.com/LArbys/lardly/blob/48efe52d8dd51bac2c0644be9ab6749f4e241b45/lardly/data/larcv_image2d.py#L8)

In [ ]:
# We import basic python utilities along  with plotly
# If you get an error saying that the plotly library is not found, you can install it via:
#  pip3 install plotly

import os,sys
import plotly as pl # import the plotly library
import plotly.graph_objects as go # import interfaces that helps us create different types of plots

# magic jupyter notebook command to reload modules
# so we can change them and get an effect without restarting the notebook
%load_ext autoreload  

In [ ]:
# Load the ROOT library
# if you see errors, you might check that
#  * did you configure the environment in the shell before calling jupyter-notebook?
#    This usually means calling, source setenv_py3.sh; source configure.sh in the top-level of the repo
#  * the build didn't complete properly. After setting the evironment above, 
#    try calling `source buildall_py3.sh` again

import ROOT as rt # You should see a message like, "Welcome to JupyROOT [version]"

In [ ]:
# load the libraries our group has written
from larlite import larlite # stores list of tabular data related an event
from larcv import larcv # stores data related to the wire plane signals represented as an image
from ublarcvapp import ublarcvapp # utility functions that interfaces between the image and tabular data
import lardly # utility functions that create default plots (3d scatter, 2d heat maps) for our data

Below we specify the location of the file we want to use.

For a list of example files, refer to this [list](https://github.com/NuTufts/icdl/wiki/Locations-of-Tutorial-Files).

In [ ]:
# Specify location of the file we want to open

# For list of tutorial files check out []
inputfile = "dlmerged_larflowtruth_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root"

In [ ]:
# open the ROOT file and use a command to list the contents
#
# you'll see a lot of TTree objects. 
# These are essentially tables of data with a specific schema, 
#   with each row of data consisting of one "event".

rfile = rt.TFile(inputfile,"open")
rfile.ls()

We do not access the data (i.e. the TTrees) directly). 

The ROOT trees store data in the form of serialized instances of custom C++ classes. 

In otherwords, for each event, copies of C++ classes are turned into a binary string, storing the values of its data members. 

Unpacking this involves de-serializing the class data and giving back a set of C++ class instances for each event. 

Not only that, but we have to coordinate the access of the trees so that the data from each is for the same event.

This is obviously complicated. We have special classes of our own that interfaces with ROOT's IO functions to help us do this. The data input/output inferface class for LArCV-type trees is `larcv::IOManager`.

If you want to look at the header, follow this [link](https://github.com/LArbys/LArCV/blob/develop/larcv/core/DataFormat/IOManager.h).

For this notebook, we are after `image2d` objects, which store the wireplane data.

In [ ]:
# We use the larcv IOManager class to load the trees for us.

# Initialize an IOManager instance that will read out input file

# There is this annoying feature left over from a code choice early on in development of LArCV.
# We initially stored images in the computer vision tradition, with the origin in the upper left.
# We later realized this was confusing and started storing images in a more physicist friendly 
#  convention which is the origin is in the lower right hange corner.
# But we still work with files made in the old way.
# So we need to specify if the images are in the old (tick backward) or new format (tick forward)

tick_order = larcv.IOManager.kTickBackward
#tick_order = larcv.IOManager.kTickForward

io = larcv.IOManager( larcv.IOManager.kREAD, "larcv", tick_order )
io.add_in_file( inputfile )

# If the format is in the old way, flip it
if tick_order==larcv.IOManager.kTickBackward:
    io.reverse_all_products()
io.initialize()



In [ ]:
# Now we load an entry in the file.
# Each entry contains one "Event".
# An event is defined as the time, we gave the order to save a picture from the detector.
# We usually do this in coordination when the neutrino beam is fired.

ENTRY_NUM = 2
io.read_entry(ENTRY_NUM)


As mentioned previously, the format of data isn't simple (copies of C++ classes), and each file holds many different types of data in different trees (see the list above).

So we have to tell the interface what type of data to load from the event.

It usually returns a contain (think list or c++ vector), with all the copies of that type of data for an event.

For the wire plane images, each event contains three images. Each image corresponds to the wire planes of the MicroBooNE liquid argon TPC.

In [ ]:
# Get the images from the tree storing the wireplane images

# we retrieve a container with all the wireplane images for the event
event_image_data = io.get_data( larcv.kProductImage2D, "wiremc" )
#event_image_data = io.get_data( larcv.kProductImage2D, "ancestor" )

# we get a std::vector<larcv::Image2D> object that has the data
adc_v = event_image_data.as_vector()

# print the number of images in the vector
# usually, if this is a MicroBooNE file you should get 3 images: one for each wireplane U, V, and Y.
print("number of images in this event: ",adc_v.size())

We loop through each image in the event and make a picture.

The wire plane images are stored in the order:
 * the U-Induction plane
 * the V-Induction plane
 * the Y-Induction plane
 
The image data is stored in a `larcv::Image2D` object. You can refer to its header file [here](https://github.com/LArbys/LArCV/blob/develop/larcv/core/DataFormat/Image2D.h) to see what data it holds and what kind of functions it has.
 
If you used the file `dlmerged_larflowtruth_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root` and accessed Entry 0, you will see a electron neutrino interaction. You can find the location of the interaction in each plane at:
 * U plane: (x,y)=(column,row)=(wire,tick)=(1648,5496)
 * V plane: (x,y)=(2230,5478)
 * Y plane: (x,y)=(3216,5508)
 
What we looking at exactly? This is the output of MicroBooNE's liquid argon time projection chamber. Specifically it is the output of its charge-sensitive wire planes.

What it is ultimately telling us is the locations in the detector where charged particles traveled and left behind energy in the detector. The portion of the energy that we can see through these images is the fraction used to create "ionization electrons".  When a charged particle passes by an argon atom, the atom feels the particles presence through it's electric charge. This charge can push on the electrons of the argon atom and ultimately liberate electrons from the argon atom in a process called "ionization". 

What the color scale shows us in the images is how many ionizations electrons were created at a given location in the detector.

OK, that's not quite the full story. What we actually see is the PROJECTED location of the ionization, not the true 3D position in the detector. We ultimately have to do some work to put the three images together (along with timing information from other pieces of data from the detector) to get the 3D location. But-- we can still look at the pictures and pick out interesting pieces of information, such as identify neutrino interactions and recognize individual particles.
 
You can see three particles being created in this interaction. The point where the three particles start is the location where the neutrino interacted with an argon nucleus. We refer to this location as the "vertex". From the vertex emerges different particles. If the particles have charge, then we can see them. Often, when we look at data, we need to run algorithms to decide what type of charged particle was created. When we don't know the name of a particle, we often refer to it (or more precisely the pattern of energy left behind) as a "prong".
 
The first is an electron which starts off as a line then quickly begins to exhibit branch-like patterns. This is because electrons create what are called "electromagnetic showers" or "showers".
 
 There are also two protons which are ejected from the nucleus that was hit by the neutrino. The first is fairly energetic. It goes off and then makes a sharp turn as the proton interacts with a second argon nucleus. The second proton is fairly low energy. Can you spot the short line of heavy energy deposition near the interaction vertex?
 

In [ ]:
# we loop through each image in the event and make a picture
# the wire plane images are stored in the order:
#  the U-Induction plane
#  the V-Induction plane
#  the Y-Induction plane

fig_v = []
for iplane in range(adc_v.size()):
    # get a copy of the image from the container
    plane_image = adc_v.at(iplane) # returns an object of type larcv::Image2D
    
    # use a lardly utility function to generate a 2D image from the larcv::Image2D object
    plane_plot = lardly.data.visualize_larcv_image2d( plane_image, reverse_ticks=False, maxz=100.0 )

    # ise plotly to show the image
    fig = go.Figure( data=[plane_plot] )
    fig.show()
    
    # put the fig in a list to keep the variable in scope
    fig_v.append(fig)